# IN2110 obligatorisk innlevering 2b

Oppgaven inneholder to deler (maskinoversettelse og interaktive systemer). Planlegg god tid slik at du klarer å fullføre de to delene av oppgaven.

Dersom du har spørsmål så kan du:

* gå på gruppetime,
* spørre på Discourse (https://in2110-discourse.uio.no/)
* eller sende epost til in2110-hjelp@ifi.uio.no dersom alternativene over av en eller annen grunn ikke passer for spørsmålet ditt.

### Oppsett
Når du har klonet dette github-repoet som denne notebooken ligger i, har du tilgang til datene som ligger i denne mappa. Hvis du ønsker å kopiere denne mappa, "2b", over til et annet sted, så skulle det gå bra. Bare pass på at du følger med på om det er oppdateringer her i repoet som gir ut obligen. Du bør også installere `transformers`, `pytorch` og `sentencepiece`:
```bash
> pip install transformers "transformers[sentencepiece]" torch
```

### Innlevering

Innleveringen skal bestå av: 
- denne Jupyter notebook fylt ut med både kode og tilhørende forklaringer.
- de to opprinnelige tekstfilene `lotr.de` og `lotr.en` som kom med obligen
- de to tekstfilene `lotr_output.en` og `lotr_corrected_en` som ble generert i Del 1 
 
Vi understreker at innlevering av koden alene __ikke er nok__ for å bestå oppgaven -- vi forventer at notebooken også skal inneholde beskrivelser (på norsk eller engelsk) av hva dere har gjort og begrunnelser for valgene dere har tatt underveis. Bruk helst hele setninger, og matematiske formler om nødvendig. Evalueringstallene bør presenteres i tabeller. Det å  forklare med egne ord (samt begreper vi har gått gjennom på forelesningene) hva dere har implementert og reflektere over hvorvidt løsningen dere har lagt  besvarer oppgaven er en viktig del av læringsprosessen -- ta det på alvor! 

# Del 1: Maskinoversettelse

Vi skal bruke en (nevral) maskinoversettelsemodell til å oversette filmtekstinger fra Ringenes Herre (og Hobbiten) fra tysk til engelsk.

## Data

Filmtekstingene ligger i filene `lotr.de` og `lotr.en` for henholdsvis de tyske og engelskspråklige filmtekstingene. Disse to filene utgjør et såkalt _parallellkorpus_, altså en tekstsamling hvor hver setning (i språk A) er koblet til en tilsvarende setning i språk B. De 2 filene har samme antall linjer, slik at den tyske setningen på linjen $i$ av `lotr.de` har en engelsk oversettelse på samme linje av `lotr.en`. Filmtekstingene er ekstrahert fra korpuset [OpenSubtitles-2018](http://opus.nlpl.eu/OpenSubtitles-v2018.php).


Her er f.eks. de 10 første linjene i `lotr.de` og `lotr.en`: 
<style scoped>
table {
  font-size: 12px;
}
</style>
|   | Tysk (`lotr.de`)         | Engelsk (`lotr.en`)      |
|---|--------------------------|--------------------------|
| 1 | Die Welt ist im Wandel . | The world is changed .   |
| 2 | Ich spüre es im Wasser . | I feel it in the water . |
| 3 | Ich spüre es in der Erde . | I feel it in the earth . |
| 4 | Ich rieche es in der Luft . | I smell it in the air . |
| 5 | Vieles , was einst war , ist verloren , da niemand mehr lebt , der sich erinnert . | Much that once was is lost . For none now live who remember it . |
| 6 | Es begann mit dem Schmieden der Großen Ringe . | It began with the forging of the Great Rings . |
| 7 | 3 wurden den Elben gegeben , den unsterblichen , weisesten und reinsten aller Wesen . | Three were given to the Elves : Immortal , wisest and fairest of all beings . |
| 8 | 7 den Zwergenherrschern , großen Bergleuten und Handwerkern in ihren Hallen aus Stein . | Seven to the Dwarf-lords : Great miners and craftsmen of the mountain halls . |
| 9 | Und 9 ... 9 Ringe wurden den Menschen geschenkt , die vor allem anderen nach Macht streben . | And nine nine rings were gifted to the race of Men who , above all else , desire power . |
| 10 | Denn diese Ringe bargen die Kraft und den Willen , jedes Volk zu leiten . | For within these rings was bound the strength and will to govern each race . |

Merk at teksten allerede er tokenisert. Noen ganger kan det være store sprik mellom innholdet i filmtekstingene. Det er ikke nødvendigvis en oversettelsefeil -- det er bare at filmtekstere kan velge å transkribere hva som skjer i filmen på litt ulike måter.

## Komme i gang

Vi skal bruke [`transformers`](https://huggingface.co/docs/transformers/index), et Python-bibliotek fra HuggingFace som gjør det lettere å kjøre nevrale modeller i NLP, blant annet for maskinoversettelse. Vi skal imidlertid ikke trene en ny modell, da trening av slike modeller krever ganske store regneressurser, blant annet GPUs. Men heldigsvis finnes det allerede pre-trente modeller. 

Vi skal benytte oss av [`opus-mt-de-en`](https://huggingface.co/Helsinki-NLP/opus-mt-de-en), som består av en _tokenizer_ og selve _seq2seq modellen_ (som baserer seg på [MarianMT](https://huggingface.co/docs/transformers/model_doc/marian)). Disse to kan lastes slik:

In [1]:
! pip install transformers "transformers[sentencepiece]" torch

  Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1


In [5]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("helsinki-nlp/opus-mt-de-en", force_downgrade=True)
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-de-en")

# Hvis du har en GPU på maskinen din kan du sette device til "cuda" 
# i stedet for "cpu", slik at oversettelsen går raskere
device = "cpu"  # "cuda"
translator = translator.to(device)

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-de-en")


Når dere kjører disse to linjene for første gang vil biblioteket automatisk laste ned modellen fra HuggingFace sin repo.

Den _tokenizer_ har ansvar for for å segmentere input-strengene i tokens og konvertere disse til tall (indeksverdier fra modellens vokabular). Den kan brukes slik:

In [8]:
tokens = tokenizer("Die Welt ist im Wandel.", return_tensors="pt", padding=True)
tokens

{'input_ids': tensor([[  55,  401,   29,   49, 9012,    3,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

Argumentet `return_tensors='pt'` brukes til å få Pytorch tensorer som resultater i stedet for vanlige lister. Merk også at vi aktiverer padding til å få de to sekvensene til å ha samme lengde, slik at de to tokensekvensene kan settes sammen i én tensor. Resultatet inkluderer både selve listen over `token IDs` og en `attention mask`. Sistnevnte består vanligvis av rekker av 1 (det vil si at modellen må ta hensyn til alle tokens) bortsett fra de kunstige "padding"-tokens som er lagt til for å få sekvenser av samme lengde. 

Deretter er det bare å bruke funksjonen `generate` til å kjøre oversettelsen (vi setter her en maks grense på 50 tokens for hver oversettelse):

In [9]:
tokens = {k:v.to(device) for k, v in tokens.items()}  # Flytt data til riktig enhet (i tilfelle man bruker GPU)
outputs = translator.generate(**tokens, max_new_tokens=50)
outputs

tensor([[58100,    36,   360,    19,  7315,     3,     0]])

Resultatet er igjen en liste over token IDs, slik at vi må konvertere disse tilbake til strenger:

In [11]:
translations =tokenizer.batch_decode(outputs, skip_special_tokens=True)
translations

['The world is changing.']

Det er selsvagt mange parametre dere kan eksperimentere hvis dere ønsker å tilpasse oversettelsestrategien (se [her](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig)), men det er egentlig alt dere trenger å vite for å bruke modellen til å oversette!

## Oversettelser

__Oppgave 1.1__: Finn ut hvordan `tokenizer` har delt opp de to setningene "Die Welt ist im Wandel." og "Ich spüre es im Wasser." i en rekke av tokens, og vis inndelingen. Hvis du ser at `tokenizer` har lagt til spesielle symboler i de 2 rekkene bør du også forklare kort hva disse står for.


Svar: 

__Oppgave 1.2__ : Bruk `opus-mt-de-en` modellen vist over til å oversette hele korpuset `lotr.de` til engelsk. Merk at du trolig må kjøre dele oversettelsen i flere _batch_ for å unngå at Python krasjer hvis minnen ikke strekker til. 

Oversettelsen kan ta litt tid avhengig av maskinen deres (og spesielt om maskinen har tilgang til en GPU). Hvis du opplever store utfordringer med regnekraft kan du redusere antall linjer på filene `lotr.de` og `lotr.en`, slik at det blir mindre å oversette.


In [17]:
# Jeg tok sjansen på å kjøre denne koden i ett sett - på tross av at PCen min ikke har GPU. Det tok en time, men gikk fint!
import re

def translate(input_file, translation_file):
    """Translate an input file line by line using the opus-mt-de-en model,
    and write the translations to output_file. The two files should
    have the same number of lines"""

    with open(input_file, 'r') as input, open(translation_file, 'w') as output:
        count = 0
        for line in input:
            tokens = tokenizer(line, return_tensors="pt", padding=True)
            tokens = {k:v.to(device) for k, v in tokens.items()}
            outputs = translator.generate(**tokens, max_new_tokens=50)
            translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            translation = re.sub(r'(?<=\w)([,.;.])', r' \1', translation)

            output.write(f'{translation}\n')
            count += 1

translate("lotr.de", "lotr_output.en")


Dere kommer sikkert til å merke at modellen ikke ble trent på tekster fra Ringenes Herre og dermed ikke oversetter navnene på personer og steder på en god måte. For eksempel er `Bilbo Baggins` egentlig oversatt til `Bilbo Beutlin` på tysk, og `the Shire` er oversatt til `Auenland`. Men det er oversettelsesystemet vårt uvitende om. 

__Oppgave 1.3__: Lag en postprosesseringsmetode som redigerer feile oversettelser for en rekke navn fra Ringenes Herre som er annerledes på tysk og på engelsk. Det er nok om du finner 10 navn som bør rettes opp, men du kan selvsagt legge til flere om du ønsker det.

In [18]:
def postprocess(translation_file, new_translation_file):
    """Edits some of the translations in translation_file to correct
    some erroneous translations due to Lord of the Rings names."""

    name_edits = {("Destiny Hill", "Fate Hill", "Fatesberg"): "Mount Doom", "Dark Ruler": "Dark Lord",
                   ("foggy mountains", "Fog mountains", "Fog Mountains"): "misty mountains", ("darling", "Honey", "treasure"): "precious", "Auenland": "Shire",
                   ("beutlin", "Beutlin"): "Baggins", "Tobi": "Toby", "Tuk": "Took", "Hobbingen": "Hobbington",
                   ("Elbish", "Elbian"): "Elvish", "Samweis": "Samwise", "Gamjie": "Gamgee", "Isengart": "Isengard",
                   ("dance pony", "dancing pony"): "Prancing Pony", "Unterberg": "Underhill", "Bruchtal": "Rivendell",
                   ("Stich"): "Sting", ("Eisenbergen"): "Iron Hills", "Hüttinger": "Cotton", "Bauer": "Farmer", "Black Horseman": "Black Rider",
                   "Herr": "Mr."}

    with open(translation_file, 'r') as input, open(new_translation_file, 'w') as output:
        for line in input:
            for key in name_edits.keys():
                if isinstance(key, tuple):
                    for name in key:
                        if name in line:
                            line = line.replace(name, name_edits[key])
                else:
                    if key in line:
                        line = line.replace(key, name_edits[key])
            output.write(line)

        

postprocess("lotr_output.en", "lotr_corrected.en")

## Evaluering

Vi er nå klare til å evaluere kvaliteten på oversettelsene vi har generert. Her skal vi bruke en evalueringsmetode som er veldig populær i maskinoversettelse, nemlig __BLEU__.  BLEU er en automatisert evalueringsmetode som sammenligner oversettelse som systemet har produsert med en eller flere fasiter, altså oversettelser skrevet av menneskelige eksperter.  I vårt tilfelle er fasiten de engelskspråklige filmtekstingene i `lotr.en`. 

BLEU beregnes ved å se på _overlapp_ mellom N-grams fra fasiten(e) og oversettelsene fra systemet. Mer presist ekstraherer vi for hver setning alle N-grams (med N fra 1 til 4) fra både systemet og fasiten, og beregner hva som er precision for $i \in {1,2,3,4}$:

\begin{equation}
    precision_i = \frac{\text{Antall $i$-grams som forekommer i både system og fasit (for samme setning)}}{\text{Antall $i$-grams i setningene fra systemet}}
\end{equation}


Deretter slår vi sammen precision-tallene:

\begin{equation}
BLEU = brevity\_penalty * \left(\prod_{i=1}^4 precision_i \right)^{\frac{1}{4}}
\end{equation}

hvor "brevity penalty" brukes til å straffe modeller som produserer for korte oversettelser:

\begin{equation}brevity\_penalty = min(1, \frac{\text{Antall ord i systemets setninger}}{\text{Antall ord i fasitens setninger}})
\end{equation}

#### Kode

Her er del av koden som kan brukes til å beregne BLEU:

In [8]:
import collections
import math
import numpy as np

def get_sentences(text_file):
    """Given a text file with one (tokenised) sentence per line, returns a list 
    of sentences , where each sentence is itself represented as a list of tokens.
    The tokens are all converted into lowercase.
    """
    
    sentences = []
    fd = open(text_file)
    for sentence_line in fd:
        # We convert everything to lowercase
        sentence_line = sentence_line.rstrip("\n").lower()
        sentences.append(sentence_line.split())
    fd.close()
    
    return sentences
    

def get_ngrams(tokens, ngram_order):
    """
    Extracts all n-grams counts of a given order from an input sequence of tokens.
    """
    ngrams = collections.Counter()
    for i in range(0, len(tokens) - ngram_order + 1):
        ngram = tuple(tokens[i:i+ngram_order])
        ngrams[ngram] += 1
    return ngrams


def compute_brevity_penalty(reference_file, output_file):
    """Computes the brevity penalty."""
    
    ref_sentences = get_sentences(reference_file)
    output_sentences = get_sentences(output_file)
    
    nb_ref_tokens = sum([len(sentence) for sentence in ref_sentences])
    nb_output_tokens = sum([len(sentence) for sentence in output_sentences])
    
    penalty = min(1, nb_output_tokens/nb_ref_tokens)
    return penalty

    
def compute_bleu(reference_file, output_file, max_order=4):
    """
    Given a reference file, an output file from the translation system, and a 
    maximum order for the N-grams, computes the BLEU score for the translations 
    in the output file.
    """
   
    precision_product = 1
    for i in range(1, max_order+1):
        precision_product *= compute_precision(reference_file, output_file, i) 
    
    brevity_penalty = compute_brevity_penalty(reference_file, output_file)
    
    bleu = brevity_penalty * math.pow(precision_product, 1/max_order)
    return bleu


In [20]:
get_sentences("lotr_corrected.en")[0]

['the', 'world', 'is', 'changing', '.']

__Oppgave 1.4__: Koden over mangler funksjonen `compute_precision(ref_file, output_file, ngram_order)` som beregner _precision_-verdien (som definert over) for en gitt N-gram ordre. Implementer denne metode.

In [21]:
def compute_precision(reference_file, output_file, ngram_order):
    """
    Computes the precision score for a given N-gram order. The first file contains the 
    reference translations, while the second file contains the translations actually
    produced by the system. ngram_order is 1 to compute the precision over unigrams, 
    2 for the precision over bigrams, and so forth.   
    """
    
    ref_sentences = get_sentences(reference_file)
    output_sentences = get_sentences(output_file)

    ngrams_match = 0
    ngrams_total = 0
    
    for sen1, sen2 in zip(ref_sentences, output_sentences):
        ref_grams = get_ngrams(sen1, ngram_order)
        out_grams = get_ngrams(sen2, ngram_order)
        for key in out_grams.keys():
            if key in ref_grams.keys():
                ngrams_match += ref_grams[key]
            ngrams_total += out_grams[key]
    
    return ngrams_match / ngrams_total

print(compute_precision("lotr.en", "lotr_output.en", 1))

0.6103174603174604


_Tips_: bruk metoden `get_ngrams(tokens, ngram_order)` som ekstraherer alle N-grams i en setning (allerede inndelt i tokens), og som allerede er implementert.

__Oppgave 1.5__: Kjør funksjonen `compute_bleu(ref_file, output_file)` på oversettelsene du har produsert. Sammenlign BLEU-resultatet du får både _med_ og _uten_ å ta i bruk postprocesseringsmetoden du har skrevet i oppgave 1.3.

In [22]:
print(compute_bleu("lotr.en", "lotr_corrected.en"))
print(compute_bleu("lotr.en", "lotr_output.en"))

0.2818837317777583
0.2776774042360547


## Del 2: Interaktive systemer

Filmtekstinger kan brukes til andre formål enn å trene og teste maskinoversettelsesystemer - filmtekstinger består også i all hovedsak av _samtaler_ og kan derfor også brukes til å bygge datadrevne dialogsystemer! 

I denne delen av oppgaven skal dere utvikle en liten _retrieval-based chatbot_ basert på korpuset i `lotr.en`. Chatboten vår vil derfor "snakke" som filmkarakterer i Ringene Herre. Hovedidéen er å:
* Beregne TF-IDF-vektorene av alle setningene i korpuset vårt og lagre disse. La oss kalle disse vektorene $[t_1, t_2, ... t_{|C|}]$, hvor $|C|$ er antall setninger i korpuset. 
* Når chatbot mottar en ny inputsetning fra brukeren beregner vi TF-IDF vektoren $q$ av denne setningen.
* Deretter leter man etter setningen i korpuset som ligner mest på inputsetningen ved å beregne _cosine similarity_ mellom TF-IDF vektoren $q$ av inputsetningen og hver TF-IDF-vektor fra korpuset $C$:

    \begin{equation}
        i^* = \argmax_{i=1}^{|C|} \frac{q^T t_i}{||q|| \ ||t_i||}
    \end{equation}

* Til slutt tar vi setningen som kommer _rett etter_ setningen $t_{i^*}$, altså $t_{i^*+1}$.

### Eksempel

For å ta et eksempel: la oss si at brukeren skriver: 

````{verbatim}
Are you Bilbo Baggins ?
````

Ifølge cosine similarity mellom TF-IDF vektorer er setningen i korpuset som ligner mest på inputsetningen på linje 4907:

````{verbatim}
Bilbo Baggins .
````

Da vil systemet ta setningen på linjen 4908 og svare brukeren:

````{verbatim}
I 'm sorry , do I know you ?
````

### Kode

Her er en del av koden som skal brukes i vår chatbot:


In [2]:
class RetrievalChatbot:
    """Retrieval-based chatbot using TF-IDF vectors"""
    
    def __init__(self, dialogue_file):
        """Given a corpus of dialoge utterances (one per line), computes the
        document frequencies and TF-IDF vectors for each utterance"""

        # I also store the dialogue file as an instance variable, to generate outputs directly from here
        self.dialogue_file = dialogue_file
        
        # We store all utterances (as lists of lowercased tokens)
        self.utterances = []
        fd = open(dialogue_file)
        for line in fd:
            utterance = self._tokenise(line.rstrip("\n"))
            self.utterances.append(utterance)
        fd.close()
        
        self.doc_freqs = self._compute_doc_frequencies()
        self.tf_idfs = [self.get_tf_idf(utterance) for utterance in self.utterances]

        
    def _tokenise(self, utterance):
        """Convert an utterance to lowercase and tokenise it by splitting on space"""
        return utterance.strip().lower().split()
    
    def _compute_doc_frequencies(self):
        """Compute the document frequencies (necessary for IDF)"""
        
        doc_freqs = {}
        for utterance in self.utterances:
            for word in set(utterance):
                doc_freqs[word] = doc_freqs.get(word, 0) + 1
        return doc_freqs

    
    def get_tf_idf(self, utterance):
        """Compute the TF-IDF vector of an utterance. The vector can be represented 
        as a dictionary mapping words to TF-IDF scores."""
         
        tf_idf_vals = {}
        word_counts = {word:utterance.count(word) for word in utterance}
        for word, count in word_counts.items():
            idf = math.log(len(self.utterances)/(self.doc_freqs.get(word,0) + 1))
            tf_idf_vals[word] = count * idf
        return tf_idf_vals
    
    
    def get_response(self, query):
        """
        Finds out the utterance in the corpus that is closed to the query
        (based on cosine similarity with TF-IDF vectors) and returns the 
        utterance following it. 
        """

        # If the query is a string, we first tokenise it
        if type(query)==str:
            query = self._tokenise(query)
        
        # Your implementation should use the get_tf_idf and compute_cosine 
        # methods that are already provided (as well as the TF-IDF values
        # from each utterance in the corpus, stored in self.tf_idfs) 
        raise NotImplementedError()
        
    
    def compute_cosine(self, tf_idf1, tf_idf2):
        """Computes the cosine similarity between two vectors"""
        
        dotproduct = 0
        for word, tf_idf_val in tf_idf1.items():
            if word in tf_idf2:
                dotproduct += tf_idf_val*tf_idf2[word]
                
        return dotproduct / (self._get_norm(tf_idf1) * self._get_norm(tf_idf2))
    
    def _get_norm(self, tf_idf):
        """Compute the vector norm"""
        
        return math.sqrt(sum([v**2 for v in tf_idf.values()]))

__Oppgave 2.1__: Fyll ut metoden get_response(self, query) som tar en brukersetning som input, og returnerer svaret som forklart ovenfor. Metoden bør ta i bruk de to metodene `get_tf_idf` og `compute_cosine` som allerede er implementert.

In [3]:
def get_response(self, query):
    """
    Finds out the utterance in the corpus that is closed to the query
    (based on cosine similarity with TF-IDF vectors) and returns the 
    utterance following it. 
    """

    # If the query is a string, we first tokenise it
    answer = "Something went wrong!"

    if type(query)==str:

        answer_index = 0
        best_cosine_similarity = -1

        query = self._tokenise(query)
        query_idf = self.get_tf_idf(query)
        for tf_idf in self.tf_idfs:
            sim = abs(self.compute_cosine(tf_idf, query_idf))
            if sim > best_cosine_similarity:
                best_cosine_similarity = sim
                answer_index = self.tf_idfs.index(tf_idf) + 1
        
        with open(self.dialogue_file) as dialogue:
            answer = dialogue.readlines()[answer_index]
        
        return answer.strip()
        
        
    # Your implementation should use the get_tf_idf and compute_cosine 
    # methods that are already provided (as well as the TF-IDF values
    # from each utterance in the corpus, stored in self.tf_idfs)

    # Implementer metoden her!

RetrievalChatbot.get_response = get_response


In [9]:
word_of_the_rings = RetrievalChatbot("lotr.en")
word_of_the_rings.get_response("Bilbo Baggins")

"I 'm sorry , do I know you ?"

Du kan deretter teste din chatbot med ulike brukerinput og se hva som kommer ut. 

In [10]:
word_of_the_rings.get_response("You shall not pass!!!!")

'Fly , you fools !'

In [15]:
word_of_the_rings.get_response("One of you must do this")

'One does not simply walk into Mordor .'